# 1. テキストデータのTF-IDFベクトル化関数
問題:
複数の商品説明文（文字列のリスト）を受け取り、scikit-learn の TfidfVectorizer を使用して各文書をTF-IDFベクトルに変換する関数 vectorize_texts_tfidf(texts) を作成してください。
ベクトル化の際には、一般的な日本語のストップワード（例: 「の」「です」「ます」など。簡単なリストで可）を除外し、トークン化は単語単位（スペース区切り、または簡易的な形態素解析ライブラリ利用を想定するが、ここでは簡易的にスペース区切りで良い）で行うこととします。
戻り値は、TF-IDF行列（疎行列または密行列）と、使用したベクトライザのインスタンスとします。

期待される動作例:

```Python
from sklearn.feature_extraction.text import TfidfVectorizer

# 簡易的な日本語ストップワードリストの例
japanese_stopwords = ["の", "に", "は", "を", "た", "が", "で", "て", "と", "し", "れ", "さ", "ある", "いる", "も", "する", "から", "な", "こと", "として", "です", "ます"]

def vectorize_texts_tfidf(texts, stopwords=japanese_stopwords):
    # ここに処理を記述
    pass

descriptions = [
    "高機能 な スマートフォン です",
    "軽量 で 高性能 な ノートパソコン",
    "この スマートフォン は 最新 です"
]
tfidf_matrix, vectorizer = vectorize_texts_tfidf(descriptions)
# tfidf_matrix は (文書数 x 特徴語数) の行列
# vectorizer.get_feature_names_out() などで特徴語を確認できる
```
注: 実際にはjanomeやMeCabといった形態素解析ライブラリと組み合わせることが望ましいですが、テストの簡略化のため上記のような想定としています。


## 回答

In [2]:
pip install scikit-learn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 簡易的な日本語ストップワードリストの例（問題文に記載されているもの）
japanese_stopwords = ["の", "に", "は", "を", "た", "が", "で", "て", "と", "し", "れ", "さ", "ある", "いる", "も", "する", "から", "な", "こと", "として", "です", "ます"]

def vectorize_texts_tfidf(texts: list[str], stopwords: list[str] = japanese_stopwords):
    """
    複数の商品説明文（文字列のリスト）を受け取り、scikit-learn の TfidfVectorizer を使用して
    各文書をTF-IDFベクトルに変換します。

    ベクトル化の際には、一般的な日本語のストップワードを除外し、トークン化は単語単位（スペース区切り）で行います。

    Args:
        texts (list[str]): 説明文のリスト。各文字列はスペースで区切られた単語と仮定します。
        stopwords (list[str], optional): 除外する日本語のストップワードのリスト。
                                         デフォルトは `japanese_stopwords` リスト。

    Returns:
        tuple: (tfidf_matrix, vectorizer_instance)
            tfidf_matrix (scipy.sparse.csr_matrix): 計算されたTF-IDF行列。
            vectorizer_instance (sklearn.feature_extraction.text.TfidfVectorizer): 使用したTfidfVectorizerのインスタンス。
    """
    # TfidfVectorizerのインスタンスを作成します。
    # stop_words: 指定されたストップワードリストを除外します。
    # tokenizer: テキストをスペースで分割してトークンを生成するカスタム関数を指定します。
    #            これにより、問題文の「簡易的にスペース区切りで良い」という要件を満たします。
    # lowercase: デフォルトTrueですが、日本語では通常変更の必要はありません。
    vectorizer = TfidfVectorizer(stop_words=stopwords, tokenizer=lambda text: text.split(' '))

    # textsの各文書をTF-IDFベクトルに変換します。
    # fit_transformメソッドは、語彙を学習し（fit）、その語彙に基づいて文書を変換（transform）します。
    tfidf_matrix = vectorizer.fit_transform(texts)

    # TF-IDF行列とベクトライザのインスタンスを返します。
    return tfidf_matrix, vectorizer

# 期待される動作例の確認
if __name__ == '__main__':
    descriptions = [
        "高機能 な スマートフォン です",
        "軽量 で 高性能 な ノートパソコン",
        "この スマートフォン は 最新 です"
    ]

    print("--- TF-IDF Vectorization Process ---")
    tfidf_matrix, vectorizer = vectorize_texts_tfidf(descriptions)

    print("\n--- TF-IDF Vectorization Results ---")
    print(f"Number of documents: {tfidf_matrix.shape[0]}")
    print(f"Number of features (vocabulary size): {tfidf_matrix.shape[1]}")

    print("\nFeatures (vocabulary after stopword removal):")
    # get_feature_names_out() で学習された特徴語（単語）のリストを取得します。
    # ここにストップワードが含まれていないことを確認できます。
    features = vectorizer.get_feature_names_out()
    print(features)

    print("\nTF-IDF Matrix (dense representation for easy viewing):")
    # 疎行列を密行列に変換して表示します。（大規模データではメモリを大量消費するため注意）
    print(tfidf_matrix.toarray())

    # 各文書とそのTF-IDFベクトルをより詳細に確認する例
    print("\nDetailed Document TF-IDF Vectors:")
    for i, doc in enumerate(descriptions):
        print(f"\nDocument {i+1}: '{doc}'")
        # 各文書のTF-IDFベクトル（疎行列の行）を密ベクトルに変換
        doc_vector = tfidf_matrix[i].toarray().flatten()
        
        # 単語とTF-IDF値のペアを表示（値が0より大きいもののみ）
        word_tfidf_scores = {word: score for word, score in zip(features, doc_vector) if score > 0}
        # スコアを降順でソートして表示
        sorted_scores = sorted(word_tfidf_scores.items(), key=lambda item: item[1], reverse=True)
        for word, score in sorted_scores:
            print(f"  {word}: {score:.4f}")

--- TF-IDF Vectorization Process ---

--- TF-IDF Vectorization Results ---
Number of documents: 3
Number of features (vocabulary size): 7

Features (vocabulary after stopword removal):
['この' 'スマートフォン' 'ノートパソコン' '最新' '軽量' '高性能' '高機能']

TF-IDF Matrix (dense representation for easy viewing):
[[0.         0.60534851 0.         0.         0.         0.
  0.79596054]
 [0.         0.         0.57735027 0.         0.57735027 0.57735027
  0.        ]
 [0.62276601 0.4736296  0.         0.62276601 0.         0.
  0.        ]]

Detailed Document TF-IDF Vectors:

Document 1: '高機能 な スマートフォン です'
  高機能: 0.7960
  スマートフォン: 0.6053

Document 2: '軽量 で 高性能 な ノートパソコン'
  ノートパソコン: 0.5774
  軽量: 0.5774
  高性能: 0.5774

Document 3: 'この スマートフォン は 最新 です'
  この: 0.6228
  最新: 0.6228
  スマートフォン: 0.4736


In [4]:
descriptions = [
    "高機能 な スマートフォン です",
    "軽量 で 高性能 な ノートパソコン",
    "この スマートフォン は 最新 です"
]
tfidf_matrix, vectorizer = vectorize_texts_tfidf(descriptions)


In [5]:
print(tfidf_matrix)
print(vectorizer)

  (0, 6)	0.7959605415681654
  (0, 1)	0.6053485081062917
  (1, 4)	0.5773502691896257
  (1, 5)	0.5773502691896257
  (1, 2)	0.5773502691896257
  (2, 1)	0.4736296010332684
  (2, 0)	0.6227660078332259
  (2, 3)	0.6227660078332259
TfidfVectorizer(stop_words=['の', 'に', 'は', 'を', 'た', 'が', 'で', 'て', 'と', 'し',
                            'れ', 'さ', 'ある', 'いる', 'も', 'する', 'から', 'な', 'こと',
                            'として', 'です', 'ます'],
                tokenizer=<function vectorize_texts_tfidf.<locals>.<lambda> at 0x13a55a1f0>)


# 練習

# 2. 簡単な商品カテゴリ分類モデルの訓練と評価
問題:
商品名（文字列のリスト X_texts）とそれに対応するカテゴリラベル（文字列のリスト y_labels）が与えられます。
これらのデータを用いて、以下の手順で簡単な商品カテゴリ分類モデルを訓練し、評価する関数 train_and_evaluate_classifier(X_texts, y_labels) を作成してください。

TfidfVectorizer を用いて商品名をベクトル化します。
データを訓練用とテスト用に分割します（例: 80%訓練、20%テスト）。
scikit-learn の LogisticRegression （または NaiveBayes）を用いて分類モデルを訓練します。
テストデータでモデルの正解率（accuracy）を計算し、返します。
期待される動作例:

```Python
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression # または from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

def train_and_evaluate_classifier(X_texts, y_labels):
    # ここに処理を記述
    pass

# サンプルデータ
X_texts = ["最新 iPhone 15 Pro", "美味しいリンゴ 青森産", "高性能ノートパソコン XYZ", "オーガニックコットン Tシャツ", "格安 Android スマホ"]
y_labels = ["スマートフォン", "食品", "PC", "衣類", "スマートフォン"]

accuracy = train_and_evaluate_classifier(X_texts, y_labels)
# accuracy は 0.0 から 1.0 の間の数値
```



## 回答

In [6]:
pip install scikit-learn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression # 指示通りLogisticRegressionを使用
from sklearn.metrics import accuracy_score

def train_and_evaluate_classifier(X_texts, y_labels):
    """
    商品名（文字列のリスト）とカテゴリラベル（文字列のリスト）を用いて、
    簡単な商品カテゴリ分類モデルを訓練し、評価する関数。

    Args:
        X_texts (list[str]): 商品名のリスト。
        y_labels (list[str]): 各商品名に対応するカテゴリラベルのリスト。

    Returns:
        float: モデルのテストデータに対する正解率（accuracy）。
    """

    # 1. TfidfVectorizer を用いて商品名をベクトル化します。
    # TfidfVectorizerのインスタンスを作成
    vectorizer = TfidfVectorizer()
    # X_texts（商品名）をTF-IDF特徴量ベクトルに変換
    X_vectorized = vectorizer.fit_transform(X_texts)

    # 2. データを訓練用とテスト用に分割します（例: 80%訓練、20%テスト）。
    X_train, X_test, y_train, y_test = train_test_split(
        X_vectorized, y_labels, test_size=0.2, random_state=42 # stratify=y_labels
    )

    # 3. scikit-learn の LogisticRegression を用いて分類モデルを訓練します。
    model = LogisticRegression(max_iter=1000)

    # 訓練データを用いてモデルを学習
    model.fit(X_train, y_train)

    # 4. テストデータでモデルの正解率（accuracy）を計算し、返します。
    # テストデータで予測を実行
    y_pred = model.predict(X_test)

    # 予測結果と実際のラベルを比較し、正解率を計算
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

In [18]:
# 期待される動作例: サンプルデータ
X_texts = ["最新 iPhone 15 Pro", "美味しいリンゴ 青森産", "高性能ノートパソコン XYZ", "オーガニックコットン Tシャツ", "格安 Android スマホ"]
y_labels = ["スマートフォン", "食品", "PC", "衣類", "スマートフォン"]

# 関数を呼び出し、結果を出力
accuracy = train_and_evaluate_classifier(X_texts, y_labels)
print(f"モデルの正解率 (Accuracy): {accuracy:.2f}")


モデルの正解率 (Accuracy): 0.00


# 練習

# 3. コサイン類似度計算関数
問題:
2つの特徴ベクトル（NumPy配列）vec1 と vec2 を受け取り、それらのコサイン類似度を計算する関数 calculate_cosine_similarity(vec1, vec2) を作成してください。
コサイン類似度は、ベクトルの内積をそれぞれのベクトルのL2ノルム（ユークリッドノルム）の積で割った値です。ゼロ除算が発生する場合は0を返すようにしてください。

期待される動作例:

```Python
import numpy as np

def calculate_cosine_similarity(vec1, vec2):
    # ここに処理を記述
    pass

vec_a = np.array([1, 1, 0, 1, 0])
vec_b = np.array([1, 0, 1, 1, 1])
vec_c = np.array([0, 0, 0, 0, 0])

similarity_ab = calculate_cosine_similarity(vec_a, vec_b) # 約0.577
similarity_ac = calculate_cosine_similarity(vec_a, vec_c) # 0.0
```


## 回答

In [5]:
import numpy as np

def calculate_cosine_similarity(vec1, vec2):
    """
    2つの特徴ベクトル（NumPy配列）のコサイン類似度を計算します。

    Args:
        vec1 (np.ndarray): 1つ目の特徴ベクトル。
        vec2 (np.ndarray): 2つ目の特徴ベクトル。

    Returns:
        float: 計算されたコサイン類似度。
    """
    
    # 1. ベクトルの内積を計算します。
    # np.dot() は2つのNumPy配列の内積を計算するのに使われます。
    dot_product = np.dot(vec1, vec2)

    # 2. それぞれのベクトルのL2ノルム（ユークリッドノルム）を計算します。
    # np.linalg.norm() はベクトルのノルムを計算するのに使われます。
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)

    # 3. ゼロ除算が発生しないようにチェックします。
    # どちらかのベクトルのノルムが0（つまりゼロベクトル）の場合、
    # そのベクトルは方向を持たないため、コサイン類似度を定義できません。
    # この問題の要件に従い、この場合は0を返します。
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0
    else:
        # 4. コサイン類似度の計算式を適用します。
        # 内積をノルムの積で割る
        cosine_similarity = dot_product / (norm_vec1 * norm_vec2)
        return float(cosine_similarity) # 念のためfloat型に変換
    


In [7]:
vec_a = np.array([1, 1, 0, 1, 0])
vec_b = np.array([1, 0, 1, 1, 1])
vec_c = np.array([0, 0, 0, 0, 0])

similarity_ab = calculate_cosine_similarity(vec_a, vec_b) # 約0.577
similarity_ac = calculate_cosine_similarity(vec_a, vec_c) # 0.0


print(similarity_ab)
print(similarity_ac)

0.5773502691896258
0.0


# 練習

# 4. K-Meansクラスタリングの適用とクラスタ中心の取得
問題:
商品データ（各商品が複数の数値特徴を持つと仮定し、NumPy配列 X_features で与えられる）とクラスタ数 k を受け取り、scikit-learn の KMeans を用いてデータをクラスタリングする関数 perform_kmeans_clustering(X_features, k) を作成してください。
この関数は、各データポイントが属するクラスタのラベル（配列）と、各クラスタの中心座標（配列）を返すものとします。

期待される動作例:

```Python
import numpy as np
from sklearn.cluster import KMeans

def perform_kmeans_clustering(X_features, k):
    # ここに処理を記述
    pass

# サンプルデータ (商品数 x 特徴数)
X_features = np.array([
    [1, 2], [1.5, 1.8], [5, 8],
    [8, 8], [1, 0.6], [9, 11]
])
k = 2
cluster_labels, cluster_centers = perform_kmeans_clustering(X_features, k)
# cluster_labels は各データ点がどのクラスタに属するかを示す配列 (例: [0, 0, 1, 1, 0, 1])
# cluster_centers は各クラスタの中心座標を示す配列 (例: [[1.16, 1.46], [7.33, 9.0]])
```


## 回答

In [1]:
import numpy as np
from sklearn.cluster import KMeans

def perform_kmeans_clustering(X_features, k):
    """
    scikit-learn の KMeans を用いでデータをクラスタリングする

    Args:
        X_features (np.ndarray): 商品データ。各行が1つの商品、各列が特徴を表す。
        形状は (n_samples, n_features)
        
    Returns:
        tpule:
            - cluster_labels (np.ndarray): 各データポイントが属するクラスタのラベル。
            形状は(n_samples, )
            - cluster_centers (np.ndarray): 各クラスタの中心座標。
            形状は(k, n_features)
    """
    # KMeansモデル
    kmeans_model = KMeans(n_clusters=k, random_state=42, n_init='auto') 
    # データにモデルを適合
    kmeans_model.fit(X_features)
    # クラスタのラベルを取得
    cluster_labels = kmeans_model.labels_
    # クラスタの中心座標を取得
    cluster_centers = kmeans_model.cluster_centers_
    
    return cluster_labels, cluster_centers



In [3]:
# サンプルデータ (商品数 x 特徴数)
X_features = np.array([
    [1, 2], [1.5, 1.8], [5, 8],
    [8, 8], [1, 0.6], [9, 11]
])
k = 2

# 関数を実行
cluster_labels, cluster_centers = perform_kmeans_clustering(X_features, k)

print("各データポイントのクラスタラベル:")
print(cluster_labels)

print("各クラスタの中心座標:")
print(cluster_centers)



各データポイントのクラスタラベル:
[1 1 0 0 1 0]
各クラスタの中心座標:
[[7.33333333 9.        ]
 [1.16666667 1.46666667]]


# 練習

# 5. 混同行列からの評価指標計算関数
問題:
ある二値分類モデルのテスト結果として得られた混同行列（2x2のNumPy配列）が与えられます。この混同行列から、正解率（Accuracy）、適合率（Precision）、再現率（Recall）、F1スコアを計算する関数 calculate_metrics_from_confusion_matrix(cm) を作成してください。
混同行列の形式は [[TN, FP], [FN, TP]] （TN: True Negative, FP: False Positive, FN: False Negative, TP: True Positive）とします。ゼロ除算が発生する場合は0を返すようにしてください。

期待される動作例:

```Python
import numpy as np

def calculate_metrics_from_confusion_matrix(cm):
    # ここに処理を記述
    pass

# 例: TN=50, FP=10, FN=5, TP=100
confusion_matrix = np.array([[50, 10], [5, 100]])
metrics = calculate_metrics_from_confusion_matrix(confusion_matrix)
# metrics は {'accuracy': accuracy_val, 'precision': precision_val, 'recall': recall_val, 'f1_score': f1_score_val} のような辞書
# 期待値: accuracy 約0.909, precision 約0.909, recall 約0.952, f1_score 約0.930
```



## 回答

In [2]:
import numpy as np

def calculate_metrics_from_confusion_matrix(cm):
    """
    混同行列から正解率、適合率、再現率、F1スコアを計算する。

    Args:
        cm (np.ndarray): 2x2のNumPy配列で表された混同行列。
                         形式は [[TN, FP], [FN, TP]]。

    Returns:
        dict: 各評価指標を含む辞書。
              {'accuracy': accuracy_val, 'precision': precision_val, 
               'recall': recall_val, 'f1_score': f1_score_val}
              ゼロ除算が発生する場合は、該当する指標の値は0となる。
    """
    
    # 混同行列から各要素を抽出
    TN = cm[0, 0]
    FP = cm[0, 1]
    FN = cm[1, 0]
    TP = cm[1, 1]

    # --- 1. 正解率 (Accuracy) の計算 ---
    # 全てのサンプル数
    total_samples = TP + TN + FP + FN
    if total_samples == 0:
        accuracy = 0.0
    else:
        accuracy = (TP + TN) / total_samples

    # --- 2. 適合率 (Precision) の計算 ---
    # 正と予測された総数
    predicted_positives = TP + FP
    if predicted_positives == 0:
        precision = 0.0
    else:
        precision = TP / predicted_positives

    # --- 3. 再現率 (Recall) の計算 ---
    # 実際に正である総数
    actual_positives = TP + FN
    if actual_positives == 0:
        recall = 0.0
    else:
        recall = TP / actual_positives

    # --- 4. F1スコア (F1 Score) の計算 ---
    # Precision と Recall の合計が0の場合（両方0の場合）もゼロ除算になるため注意
    if precision + recall == 0:
        f1_score = 0.0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)
    
    # 結果を辞書にまとめて返す
    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score
    }
    
    return metrics



In [3]:
# --- 期待される動作例の検証 ---
print("--- 例1 ---")
# 例: TN=50, FP=10, FN=5, TP=100
confusion_matrix1 = np.array([[50, 10], [5, 100]])
metrics1 = calculate_metrics_from_confusion_matrix(confusion_matrix1)
print(f"混同行列:\n{confusion_matrix1}")
print(f"計算結果: {metrics1}")
# 期待値: accuracy 約0.909, precision 約0.909, recall 約0.952, f1_score 約0.930
# 実際: {'accuracy': 0.9090909090909091, 'precision': 0.9090909090909091, 'recall': 0.9523809523809523, 'f1_score': 0.9302325581395349}


# --- その他のテストケース ---
print("\n--- 全ての要素が0の混同行列 ---")
# TN=0, FP=0, FN=0, TP=0
confusion_matrix_all_zeros = np.array([[0, 0], [0, 0]])
metrics_all_zeros = calculate_metrics_from_confusion_matrix(confusion_matrix_all_zeros)
print(f"混同行列:\n{confusion_matrix_all_zeros}")
print(f"計算結果: {metrics_all_zeros}")
# 期待値: {'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0}

print("\n--- 完全な分類（TPのみ） ---")
# TN=0, FP=0, FN=0, TP=100
confusion_matrix_perfect_tp = np.array([[0, 0], [0, 100]])
metrics_perfect_tp = calculate_metrics_from_confusion_matrix(confusion_matrix_perfect_tp)
print(f"混同行列:\n{confusion_matrix_perfect_tp}")
print(f"計算結果: {metrics_perfect_tp}")
# 期待値: {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}

print("\n--- 全て負と予測され、実際も全て負（TNのみ） ---")
# TN=100, FP=0, FN=0, TP=0
confusion_matrix_perfect_tn = np.array([[100, 0], [0, 0]])
metrics_perfect_tn = calculate_metrics_from_confusion_matrix(confusion_matrix_perfect_tn)
print(f"混同行列:\n{confusion_matrix_perfect_tn}")
print(f"計算結果: {metrics_perfect_tn}")
# 期待値: {'accuracy': 1.0, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0}

print("\n--- 全て誤って正と予測（FPのみ） ---")
# TN=0, FP=10, FN=0, TP=0
confusion_matrix_only_fp = np.array([[0, 10], [0, 0]])
metrics_only_fp = calculate_metrics_from_confusion_matrix(confusion_matrix_only_fp)
print(f"混同行列:\n{confusion_matrix_only_fp}")
print(f"計算結果: {metrics_only_fp}")
# 期待値: {'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0}

print("\n--- 全て誤って負と予測（FNのみ） ---")
# TN=0, FP=0, FN=5, TP=0
confusion_matrix_only_fn = np.array([[0, 0], [5, 0]])
metrics_only_fn = calculate_metrics_from_confusion_matrix(confusion_matrix_only_fn)
print(f"混同行列:\n{confusion_matrix_only_fn}")
print(f"計算結果: {metrics_only_fn}")
# 期待値: {'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0}

--- 例1 ---
混同行列:
[[ 50  10]
 [  5 100]]
計算結果: {'accuracy': 0.9090909090909091, 'precision': 0.9090909090909091, 'recall': 0.9523809523809523, 'f1_score': 0.9302325581395349}

--- 全ての要素が0の混同行列 ---
混同行列:
[[0 0]
 [0 0]]
計算結果: {'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0}

--- 完全な分類（TPのみ） ---
混同行列:
[[  0   0]
 [  0 100]]
計算結果: {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}

--- 全て負と予測され、実際も全て負（TNのみ） ---
混同行列:
[[100   0]
 [  0   0]]
計算結果: {'accuracy': 1.0, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0}

--- 全て誤って正と予測（FPのみ） ---
混同行列:
[[ 0 10]
 [ 0  0]]
計算結果: {'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0}

--- 全て誤って負と予測（FNのみ） ---
混同行列:
[[0 0]
 [5 0]]
計算結果: {'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0}


# 練習

# 6. 特徴量の標準化関数
問題:
数値特徴からなるデータセット（NumPy配列 X、各行がサンプル、各列が特徴量）を受け取り、各特徴量を標準化（平均0、標準偏差1にスケーリング）する関数 standardize_features(X) を作成してください。scikit-learn の StandardScaler を使用するか、NumPyで直接計算しても構いません。

期待される動作例:

```Python
import numpy as np
# from sklearn.preprocessing import StandardScaler # 使っても良い

def standardize_features(X):
    # ここに処理を記述
    pass

X_original = np.array([[1, -1, 2],
                       [2, 0, 0],
                       [0, 1, -1]], dtype=float)
X_standardized = standardize_features(X_original)
# X_standardized の各列は平均がほぼ0、標準偏差がほぼ1になる
# 例:
# [[ 0.         -1.22474487  1.33630621]
#  [ 1.22474487  0.         -0.26726124]
#  [-1.22474487  1.22474487 -1.06904497]]
```


## 回答

# 練習

# 7. LightGBM/XGBoostの主要ハイパーパラメータ設定の理解
問題:
LightGBMまたはXGBoostのような勾配ブースティング木モデルにおいて、モデルの複雑さを制御し、過学習を抑制するために調整されることが多い主要なハイパーパラメータを3つ挙げ、それぞれのパラメータがモデルにどのような影響を与えるかを簡単に説明してください。そして、それらのパラメータを指定してモデルを初期化する（訓練は不要）簡単なPythonコードスニペットを lightgbm.LGBMClassifier または xgboost.XGBClassifier を用いて示してください。

期待される説明とコード例:

例として挙げるパラメータ:
n_estimators (木の数): 増やすとモデルの表現力は上がるが、過学習しやすくなる。
learning_rate (学習率): 小さいほど学習は慎重に進み、汎化性能が上がることがあるが、多くの木が必要になる。
max_depth (木の深さ): 深いほど複雑な関係を学習できるが、過学習しやすくなる。
（その他、num_leaves, min_child_samples, subsample, colsample_bytree なども候補）

```Python
# LightGBMの場合の例
import lightgbm as lgb

def initialize_lgbm_with_params(n_estimators, learning_rate, max_depth):
    model = lgb.LGBMClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        random_state=42 # 再現性のため
    )
    return model

# 使用例
lgbm_model = initialize_lgbm_with_params(n_estimators=100, learning_rate=0.1, max_depth=5)
print(lgbm_model.get_params())
```

注: この問題はコーディングそのものより、ハイパーパラメータの知識を問うものです。

## 回答

# 練習

# 8. 不均衡データに対する簡易オーバーサンプリング関数
問題:
二値分類タスクにおいて、少数派クラスのデータが極端に少ない不均衡データセット（特徴量 X とラベル y、NumPy配列）が与えられたとします。少数派クラスのサンプルを単純に複製することでオーバーサンプリングを行う関数 simple_oversample(X, y) を作成してください。多数派クラスのサンプル数は変更せず、少数派クラスのサンプル数が多数派クラスのサンプル数と同じになるように複製するものとします。
（imblearn ライブラリの RandomOverSampler のような高度なものではなく、基本的な動作を実装してください。）

期待される動作例:

```Python
import numpy as np
from collections import Counter

def simple_oversample(X, y):
    # ここに処理を記述
    pass

# サンプルデータ (特徴量は1次元で簡略化)
X_imbalanced = np.array([[1],[2],[3],[4],[5],[6],[7],[8],[9],[10]])
y_imbalanced = np.array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0]) # クラス1が少数派

X_resampled, y_resampled = simple_oversample(X_imbalanced, y_imbalanced)
# Counter(y_resampled) の結果、両クラスのサンプル数が等しくなる (この例ではクラス0が8件、クラス1が2件なので、クラス1を6件複製し合計16件、各8件)
# X_resampled の形状と y_resampled の形状も確認
```


## 回答

# 練習

# 9. 商品レビューの簡易感情分析（ポジネガ辞書利用）
問題:
商品レビューのテキストと、ポジティブ単語のセット positive_words、ネガティブ単語のセット negative_words を受け取り、レビューの感情スコアを計算する関数 calculate_sentiment_score(review_text, positive_words, negative_words) を作成してください。
スコアは「レビューに含まれるポジティブ単語の数 - レビューに含まれるネガティブ単語の数」とします。単語のカウントは単純な出現回数とし、大文字・小文字は区別しないものとします。

期待される動作例:

```Python
def calculate_sentiment_score(review_text, positive_words, negative_words):
    # ここに処理を記述
    pass

positive_set = {"良い", "素晴らしい", "満足", "最高", "便利"}
negative_set = {"悪い", "残念", "不満", "ひどい", "壊れた"}

review1 = "この商品はとても良い。素晴らしい！"
score1 = calculate_sentiment_score(review1, positive_set, negative_set) # 期待値: 2 (良い, 素晴らしい)

review2 = "期待外れで残念。少し悪い点もある。"
score2 = calculate_sentiment_score(review2, positive_set, negative_set) # 期待値: -2 (残念, 悪い)

review3 = "特にコメントなし。"
score3 = calculate_sentiment_score(review3, positive_set, negative_set) # 期待値: 0
```


## 回答

# 練習

# 10. 時系列データからの移動平均計算関数
問題:
商品の売上データなどの時系列データ（数値のリストまたはNumPy配列 series）と、ウィンドウサイズ window_size を受け取り、移動平均を計算してリストまたはNumPy配列で返す関数 calculate_moving_average(series, window_size) を作成してください。
計算結果の配列の長さは、元の時系列データから window_size - 1 を引いたものになります（ウィンドウ内のデータが揃わない先頭部分は計算しない）。

期待される動作例:

```Python
import numpy as np

def calculate_moving_average(series, window_size):
    # ここに処理を記述
    pass

sales_data = np.array([10, 12, 11, 15, 16, 14, 18, 20, 19, 22])
window = 3
moving_avg = calculate_moving_average(sales_data, window)
# 期待される出力 (numpy array): [11.        , 12.66666667, 14.        , 15.        , 16.        , 17.33333333, 19.        , 20.33333333]
# ( (10+12+11)/3, (12+11+15)/3, ... )
```


## 回答

# 練習